# Демонстрация TF-IDF

Импортируем необходимые библиотеки

In [ ]:
import pandas as pd
from IPython.display import display, Math

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import numpy as np
import scipy.sparse as sp
import nltk
from nltk.text import TextCollection

Инициализируем массив ```texts```, который содержит в себе 3 элемента - 3 'синтетических' текста

In [ ]:
texts = [u'мама мама мама мыла рама',
         u'рама это рама все просто',
         u'очень просто']

# Считаем частотность

Без использования дополнительных библиотек посчитаем частотность каждого слова по всем текстам

In [ ]:
word_to_number = {}
for text in texts:
    for word in text.split():
        word_to_number[word] = word_to_number.get(word, 0) + 1

Для красивой визуализации используем pandas, не обязательно, но наглядно

In [ ]:
pd.DataFrame(list(word_to_number.items()))

Без использования дополнительных библиотек посчитаем частотность каждого слова по каждому тексту

In [ ]:
word_to_number_for_each_texts = {}
for text in texts:
    word_to_number_for_each_texts[text] = {}
    for word in text.split():
        if word_to_number_for_each_texts[text].get(word):
            word_to_number_for_each_texts[text][word] += 1
        else:
            word_to_number_for_each_texts[text][word] = 1

In [ ]:
word_to_number_for_each_texts

Для красивой визуализации использум pandas, не обязательно, но наглядно (в этот раз для красивой визуализации нужно заморочиться, но мы не будем заморачиваться)

In [ ]:
pd.DataFrame(list(word_to_number_for_each_texts.items()))

In [ ]:
pd.DataFrame(list(word_to_number_for_each_texts.values()))

Если слова нет в тексте, получаем NaN. Заменим такие значения нулями.

In [ ]:
pd.DataFrame(list(word_to_number_for_each_texts.values())).fillna(0)

А теперь воспользуемся библиотекой ```sklearn```

In [ ]:
count_vect = CountVectorizer()
temp_matrix = count_vect.fit_transform(texts) # temp_matrix эта промежуточная матрица, понадобится в следующем кейсе,
                                              # для вычисления tf-idf
matrix_counts = temp_matrix.toarray()         # в данной матрице хранятся 

In [ ]:
pd.DataFrame(temp_matrix.toarray())

In [ ]:
sorted(list(count_vect.vocabulary_.items()), key=lambda x: x[1])

Визуализируем красиво ```matrix_counts```

In [ ]:
words = [x[0] for x in sorted(count_vect.vocabulary_.items(), key=lambda x: x[1])] # список слов, 
                                                                                   # чтобы сделать красивую шапку
pd.DataFrame(matrix_counts, columns=words)  # при создании DataFrame передадим подготовленный список слов

# Что такое TF-IDF?

## TF - term frequency

1) Просто частотность, что мы считали выше (https://en.wikipedia.org/wiki/Tf–idf)

In [ ]:
display(Math(r'f_{t,d}'))

2) TF по версии российской википедии (https://ru.wikipedia.org/wiki/TF-IDF) (!!!)

In [ ]:
display(Math(r'\mathrm{tf}(t,d) = \frac{f_{t,d}}{\sum_k f_{t_k,d}}'))

3) Бинарная встречаемость. *0 - не было слова в тексте, 1 - было слово в тексте*

In [ ]:
display(Math(r'\mathrm{tf}(t,d) = \left\{ \begin{matrix} 0, f_{t,d} = 0 \\ 1, f_{t,d} ≥ 1 \end{matrix} \right.'))

4) Логарифм от частоты +1

In [ ]:
display(Math(r'\mathrm{tf}(t,d) = \log(1 + f_{t,d})'))

5) Нормированная частота (double normalization)

In [ ]:
display(Math(r'K + (1 - K) \frac { f_{t,d} }{\max({f_{t_i,d}})}'))

Подробности https://en.wikipedia.org/wiki/Tf–idf

## IDF - inverse document frequency

1) 'Стандарт', логарифм отношения числа всех документов к числу документов, содержащих данное слово

In [ ]:
display(Math(r'\mathrm{idf}(t,D)=\log\frac{N}{|\{d\in D:t\in d\}|}\
             =\log\frac{|D|}{|(d_{i}\supset t_{i})|}=\log\frac{N}{n_t}'))

2) Сглаженный IDF

In [ ]:
display(Math(r'\mathrm{idf}(t,D)=\log(1+\frac{N}{n_t})'))

3) IDF с использованием максимального значения DF

In [ ]:
display(Math(r'\log\left(1 + \frac {\max_t n_t} {n_t}\right)'))

4) Вероятностная обратная встречаемость

In [ ]:
display(Math(r'\log\frac {N - n_t} {n_t}'))

# Вычислим TF-IDF

## sklearn

Документация http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer

Исходный код https://github.com/scikit-learn/scikit-learn/blob/bb592f3865f02f1d6bf9dedce1a2554fa0ada800/sklearn/feature_extraction/text.py#L901

In [ ]:
display(Math(r'smooth\_idf == True: idf = log(\frac{N+1}{df + 1}) + 1'))
display(Math(r'smooth\_idf == False: idf = log(\frac{N}{df}) + 1'))

In [ ]:
tfidf_transformer = TfidfTransformer()
matrix_tfidf = tfidf_transformer.fit_transform(temp_matrix).toarray()
df_index = pd.DataFrame(matrix_tfidf, columns=words)
df_index

In [ ]:
pd.DataFrame([tfidf_transformer.idf_]*3, columns=words)  # построим только ```idf```

# Поисковое ранжирование
Ну посчитали мы tf-idf, что с ним делать теперь? Например, это можно использовать как значение релевантности в поиске

In [ ]:
import nltk

In [ ]:
# nltk.download('punkt')

In [ ]:
query = 'просто мама'
q_words = nltk.word_tokenize(query, language='russian')

In [ ]:
q_words

In [ ]:
[df_index.loc[0][wrd] for wrd in q_words]

In [ ]:
arr =[np.asarray([df_index.loc[i][wrd] for wrd in q_words]).sum() for i in range(len(df_index))]
pd.DataFrame(data = np.asarray([texts, arr]).T, columns = ['text', 'rel']).sort_values(by='rel', ascending=False)

А если база посерьезнее? Поищем что-нибудь про Родю Раскольникова :)
Каждый абзац у нас будет считаться отдельным документом

In [ ]:
with open('dostoevsky.txt', encoding='cp1251') as f:
    book = f.readlines()

In [ ]:
count_vect = CountVectorizer()
temp_matrix = count_vect.fit_transform(book) # temp_matrix эта промежуточная матрица, понадобится в следующем кейсе,
                                              # для вычисления tf-idf
matrix_counts = temp_matrix.toarray()         # в данной матрице хранятся 
matrix_counts

In [ ]:
book[0]

In [ ]:
matrix_counts[0].sum()

In [ ]:
temp_matrix.shape

In [ ]:
tfidf_transformer = TfidfTransformer()
matrix_tfidf = tfidf_transformer.fit_transform(temp_matrix.toarray()).toarray()
words = [x[0] for x in sorted(count_vect.vocabulary_.items(), key=lambda x: x[1])] # список слов, 

df_index = pd.DataFrame(matrix_tfidf, columns=words)
df_index.head()

In [ ]:
query = 'Родион подумал'
q_words = nltk.word_tokenize(query.lower(), language='russian')
q_words

In [ ]:
arr =[np.asarray([df_index.loc[i][wrd] for wrd in q_words]).sum() for i in range(len(df_index))]
result = pd.DataFrame(data = np.asarray([book, arr]).T, columns = ['text', 'rel']).sort_values(by='rel', ascending=False)

#result['rel'] = result['rel'].apply(lambda x: float(x))
result['rel'] = result['rel'].astype(float)
result = result[result['rel'] > 0]
result.head()

In [ ]:
len(result)

In [ ]:
result.iloc[0, 1]